# Workflow Example with ERA5 input convertor
Here we illustrate a complete workflow example using the ERA5 convertor

## Imports

In [ ]:
from pathlib import Path

import xarray as xr
import pandas as pd

import valenspy as vp #The Valenspy package
from valenspy.inputconverter_functions import _non_convertor, ERA5_to_CF

from yaml import safe_load
from typing import Union, List

In [ ]:
#Import Converter - This input converter will not do anything to the data.
ic = vp.InputConverter(ERA5_to_CF)


## 1. Convert the data

### Get the filenames of ERA5 data corresponding to CORDEX variable name


In [ ]:
machine = "hortense"

# User defined variable, here we look at 2m temperature. 
variable = "tas"
dataset = "ERA5"
region = "europe" # "belgium"
time_freq = "daily" ## Important to account here are the postprocessed files at daily time resolution and their naming (eg max daily temperature does not correspond to ERA5 variable name as defined!!)

### Generate the path and filename of obs file

In [ ]:
# Get the current file directory and load the CORDEX variables.yml file

# files = Path(__file__).resolve().parent -- this is not working in notebook
src_path = Path("../src/valenspy") ## -- to be removed. 

with open(src_path / "ancilliary_data" / "CORDEX_variables.yml") as file:
    CORDEX_VARIABLES = safe_load(file)

with open(src_path / "ancilliary_data" / Path("ERA5_lookup.yml")) as file:
    obs_LOOKUP = safe_load(file)

In [ ]:
# this will be part of the PATH generator. 

# get path of observational datasets from path settings .yml
with open(src_path / "ancilliary_data" / Path("dataset_PATHS.yml")) as file:
    dataset_PATHS = safe_load(file)
directory = dataset_PATHS[machine][dataset]

# get ERA5 variable corresponding to the requested variable using its look-up table
obs_var = obs_LOOKUP[variable]['obs_name']

# get ERA5 subdirectory
subdirectory = Path(directory + '/' + region +'/'+time_freq+'/'+obs_LOOKUP[variable]['obs_long_name']+'/')

# open the EOBS file for the corresponding variable
files = list(subdirectory.glob("*-"+time_freq+"-*"+obs_LOOKUP[variable]['obs_long_name']+"*20*.nc")) #Select all the netCDF files in the directory
files

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2013.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2010.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2017.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2006.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2014.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2019.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/

In [ ]:
# test of ERA5 convertor function
ERA5_ds = ic.convert_input(files) #Convert the input to the correct format
ERA5_ds

The file is ValEnsPy CF compliant.
50.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['tas']
Unknown to ValEnsPy: ['time_bnds']


<xarray.Dataset> Size: 3GB
Dimensions:    (time: 7305, bnds: 2, lat: 163, lon: 289)
Coordinates:
  * time       (time) datetime64[ns] 58kB 2000-01-01T11:30:00 ... 2019-12-31T...
  * lon        (lon) float32 1kB -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * lat        (lat) float32 652B 73.5 73.25 73.0 72.75 ... 33.5 33.25 33.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 117kB dask.array<chunksize=(366, 2), meta=np.ndarray>
    tas        (time, lat, lon) float64 3GB dask.array<chunksize=(362, 161, 286), meta=np.ndarray>
Attributes:
    CDI:                 Climate Data Interface version 1.9.5 (http://mpimet....
    Conventions:         CF-1.6
    history:             Tue Feb 08 16:00:46 2022: cdo -z zip daymean /mnt/HD...
    frequency:           day
    CDO:                 Climate Data Operators version 1.9.5 (http://mpimet....
    dataset:             ERA5
    spatial_resolution:  0.1deg
    freq:                europe
    region:              2m_temperature

In [7]:
ERA5_ds['tas']

<xarray.DataArray 'tas' (time: 7305, lat: 163, lon: 289)> Size: 3GB
dask.array<concatenate, shape=(7305, 163, 289), dtype=float64, chunksize=(362, 161, 286), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 58kB 2000-01-01T11:30:00 ... 2019-12-31T11...
  * lon      (lon) float32 1kB -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * lat      (lat) float32 652B 73.5 73.25 73.0 72.75 ... 33.75 33.5 33.25 33.0
Attributes:
    long_name:           Near-Surface Air Temperature
    units:               K
    standard_name:       air_temperature
    original_name:       t2m
    original_long_name:  2m_temperature
    freq:                europe
    region:              2m_temperature
    dataset:             ERA5
    spatial_resolution:  0.1deg

In [8]:
#Multiple variables test.

variables = ["tas", "pr"]
files=[]
for variable in variables:
    # get ERA5 subdirectory
    subdirectory = Path(directory + '/' + region +'/'+time_freq+'/'+obs_LOOKUP[variable]['obs_long_name']+'/')
    files+=list(subdirectory.glob("*-"+time_freq+"-*"+obs_LOOKUP[variable]['obs_long_name']+"*20*.nc")) #Select all the netCDF files in the directory
files

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2013.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2010.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2017.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2006.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2014.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/daily/2m_temperature/era5-daily-europe-2m_temperature-2019.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/

In [10]:
ERA5_ds = ic.convert_input(files) #Convert the input to the correct format
ERA5_ds['tas']

The file is ValEnsPy CF compliant.
66.67% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['tas', 'pr']
Unknown to ValEnsPy: ['time_bnds']


<xarray.DataArray 'tas' (time: 7305, lat: 163, lon: 289)> Size: 3GB
dask.array<concatenate, shape=(7305, 163, 289), dtype=float64, chunksize=(362, 161, 286), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 58kB 2000-01-01T11:30:00 ... 2019-12-31T11...
  * lon      (lon) float32 1kB -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * lat      (lat) float32 652B 73.5 73.25 73.0 72.75 ... 33.75 33.5 33.25 33.0
    units    <U10 40B 'kg m-2 s-1'
Attributes:
    long_name:           Near-Surface Air Temperature
    units:               K
    standard_name:       air_temperature
    original_name:       t2m
    original_long_name:  2m_temperature
    freq:                europe
    region:              2m_temperature
    dataset:             ERA5
    spatial_resolution:  0.1deg